In [11]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import torch.nn as nn 
import os
from matplotlib import pyplot as plt
from skimage import io, transform
import numpy as np
from torch.utils.data import DataLoader


In [12]:
class ImageNetDataset(Dataset):
    
    def __init__(self, root_dir, data_folder, transform=None):
        self.data_path = os.path.join(root_dir,data_folder)
        self.data = os.listdir(self.data_path)
        self.root = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_name = self.data[index]
        img_path = os.path.join(self.data_path,img_name)
        image = io.imread(img_path)
        if self.transform:
            image = self.transform(image)
        
        return image,img_name

In [13]:
class addGaussian:
    
    def __init__(self, std_range):
        self.std_range = std_range

    def __call__(self,img):
        minval,maxval = self.std_range
        noise_img = img.astype(np.float)
        stddev = np.random.uniform(minval, maxval)
        noise = np.random.randn(*img.shape) * stddev
        noise_img += noise
        noise_img = np.clip(noise_img, 0, 255).astype(np.uint8)
        return noise_img

In [14]:
class Rescale(object):
    
    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        image = sample
        
        new_h, new_w = self.output_size

        img = transform.resize(image, (new_h, new_w))

        return img


In [27]:
trfmd_dataset = ImageNetDataset('/home/turing/Documents/BE/','data/', 
                               transform=transforms.Compose([
                                   addGaussian((0,50)),
                                   Rescale((256,256)),
#                                    transforms.ToTensor()
                               ]))

In [28]:
trainloader = DataLoader(trfmd_dataset, batch_size=4, shuffle=True, num_workers=4)

In [18]:
num_epochs = 200
batch_size = 128
learning_rate = 1e-3

In [19]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.encoder = nn.Sequential(
            
            # first and second layer.
            # second layer
            nn.Conv2d(3,48,3),
            nn.LeakyReLU(True),            
            nn.Conv2d(48,48,3),
            nn.LeakyReLU(True),
            nn.MaxPool2d(2,2),
            
            # third conv
            nn.Conv2d(48,48,3),
            nn.LeakyReLU(True),
            nn.MaxPool2d(2,2),
            
            # fourth convolve
            nn.Conv2d(48,48,3),
            nn.LeakyReLU(True),
            nn.MaxPool2d(2,2),

            # fifth convolve
            nn.Conv2d(48,48,3),
            nn.LeakyReLU(True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(48,48,3),
            nn.LeakyReLU(True)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(48,96,3),
            nn.LeakyReLU(True),
            
            nn.ConvTranspose2d(96,96,3),
            nn.LeakyReLU(True),
            
            nn.ConvTranspose2d(96,96,3),
            nn.LeakyReLU(True),
            
            nn.ConvTranspose2d(96,96,3),
            nn.LeakyReLU(True),
            
            nn.ConvTranspose2d(96,96,3),
            nn.LeakyReLU(True),

            nn.ConvTranspose2d(96,96,3),
            nn.LeakyReLU(True),
            
            nn.ConvTranspose2d(96,96,3),
            nn.LeakyReLU(True),

            nn.ConvTranspose2d(96,96,3),
            nn.LeakyReLU(True),
            
            nn.ConvTranspose2d(96,64,3),
            nn.LeakyReLU(True),
            
            nn.ConvTranspose2d(64,32,3),
            nn.LeakyReLU(True),
            
            
            nn.Conv2d(32,3,3),
        )
        
        def forward(self, x):
            x = self.encoder(x)
            x = self.decoder(x)
            
            return x